In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [5]:
cv2.__version__

'4.10.0'

In [15]:

## [initialize_FaceDetectorYN]
detector = cv2.FaceDetectorYN.create(
    "model/face_detection_yunet_2022mar.onnx",
    "",
    (320, 320),
    0.9,
    0.3,
    5000
)



In [21]:

image = cv2.imread("face_img/akhavan.jpg")
height, width, _ = image.shape

# Set input size before inference
detector.setInputSize((width, height))
#
result = detector.detect(image)

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\net_impl.cpp:279: error: (-204:Requested object was not found) Layer with requested id=-1 not found in function 'cv::dnn::dnn4_v20240521::Net::Impl::getLayerData'
